https://blog.csdn.net/qq_41897800/article/details/114333091

torch里面的Transformer库要怎么用？例子如下：
https://github.com/CVxTz/time_series_forecasting/blob/26de501ec28bab1153a8a7abb2ad664d446ada57/time_series_forecasting/model.py#L69

看了一些实现，发现torch自带的transformer encoder decoder是没有pad，mask这样的功能的，得我们自己加进去。tnnd，这不误事嘛。

https://zhuanlan.zhihu.com/p/353365423 关于一些mask要怎么用。

In [1]:
import numpy as np      # tang.npz的压缩格式处理
import os       # 打开文件
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchnet import meter
import tqdm, math

In [2]:
def get_data():
    if os.path.exists(data_path):
        datas = np.load(data_path, allow_pickle=True)      # 加载数据
        data = datas['data']  # numpy.ndarray
        word2ix = datas['word2ix'].item()   # dic
        ix2word = datas['ix2word'].item()  # dic
        return data, word2ix, ix2word

In [3]:
def get_attn_pad_mask(dt):
    '''
    给出一个pad的mask，就是把pad符号位置标为true。
    '''
    pad_attn_mask = dt.data.eq(8292)
    return pad_attn_mask

def replace_start_with_pad(dt):
    '''
    就是把start符号转为pad符号。
    '''
    return dt.masked_fill_(dt.eq(8291), 8292)

def move_pad_to_tail(dt_ori):
    '''
    本来的数据是pad填充在前面，这个函数可以把pad挪移到每一句的后面。
    '''
    dt = dt_ori.clone() ## https://zhuanlan.zhihu.com/p/344458484
    for i in range(dt.shape[0]):
        idx = int(torch.nonzero(dt[i]==8291).squeeze())
        new_line = torch.cat(
            (dt[i][idx:], dt[i][:idx]), 0
        )
        dt[i] = new_line
    return dt

In [4]:
def gen_trg_mask(length, device = "cpu"):
    mask = torch.tril(torch.ones(length, length, device=device)) == 1

    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )

    return mask

def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
    r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
        Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [5]:
data_path = '../originalDataset/tang.npz'
data, word2ix, ix2word = get_data()
data = torch.from_numpy(data)
dataloader = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱

In [6]:
data.shape

torch.Size([57580, 125])

In [7]:
def print_origin_poem(i):
    '''
    data里面每一行就是一首诗，总共57580首。
    本函数可以打印出data[i]对应的诗。
    '''
    poem_txt = [ix2word[int(i)] for i in data[i]]
    print("".join(poem_txt[poem_txt.index("<START>") + 1:-1]))
    print()
    print("".join(poem_txt[:]))
    print()
    print(data[i])
print_origin_poem(45423)

缓步出居处，过原边雁行。夕阳投草木，远水迎苍茫。野寺同蟾宿，云溪劚药尝。萧条霜景暮，极目尽堪伤。

</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>缓步出居处，过原边雁行。夕阳投草木，远水迎苍茫。野寺同蟾宿，云溪劚药尝。萧条霜景暮，极目尽堪伤。<EOP>

tensor([8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292, 8292,
        8292, 8292, 8292, 8291,  239, 6018, 6673, 5951, 6144, 7066,  351, 1897,
        3754, 5642, 7905, 74

In [8]:
## 看看总共循环多少次
for dt in tqdm.tqdm(dataloader):
    print(dt.shape)
    print(dt.transpose(1, 0).shape)
    break

  0%|                                                  | 0/5758 [00:01<?, ?it/s]

torch.Size([10, 125])
torch.Size([125, 10])


8291是start，8290是end，8292是pad

In [9]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2, batch_first=False)
        # lstm输入为：seq, batch, input_size
        # lstm输出为：seq * batch * 256; (2 * batch * 256,...)
        self.linear1 = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        seq_len, batch_size = input.size()
        if hidden is None:
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            h_0, c_0 = Variable(h_0), Variable(c_0)
        else:
            h_0, c_0 = hidden
        embeds = self.embeddings(input)     # (seq_len, batch_size, embedding_dim), (124,128,128) ## 难道不应该是(124, 10, 128)
        output, hidden = self.lstm(embeds, (h_0, c_0))      #(seq_len, batch_size, hidden_dim), (124,128,256)
        output = self.linear1(output.view(seq_len*batch_size, -1))      # ((seq_len * batch_size),hidden_dim), (15872,256) → (15872,8293)
        return output, hidden


In [10]:
import opencc, re
cc = opencc.OpenCC('t2s')
cc.convert("秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉余。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。")

'秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉余。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。'

In [11]:
def preprocess_sentence_cn(w):
  #将繁体字转换为简体字  
    w = cc.convert(w)
    w = ' '.join(list(w))
    w = re.sub(r'[" "]+', " ", w)
    w = w.strip().rstrip()
    w = '<START> ' + w + ' <EOP>'
    new_list = []
    for i in w.split():
        new_list.append(
            word2ix.get(i, 8292)
        )
    return new_list

In [12]:
import io
def create_dataset(path):
    lines = io.open(path,encoding='utf8').read().strip().split('\n')
   
    sentence_pairs = [[preprocess_sentence_cn(w) for w in line.replace("，","。").replace("。", " ").strip().split(' ')] for line in lines]
    # print(sentence_pairs)
    return zip(*sentence_pairs)

a,b = create_dataset('../originalDataset/poem5.txt')

In [13]:
a, b = np.array(list(a)), np.array(list(b))

In [14]:
dataloader_1 = torch.utils.data.DataLoader(a, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱
dataloader_2 = torch.utils.data.DataLoader(b, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱

In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class Trsfmr_encoder_decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, nheads = 8, nlayers=6):
        
        super(Trsfmr_encoder_decoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.poc_enc = PositionalEncoding(self.embedding_dim)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nheads) # , dim_feedforward=hidden_dim
        self.transformer_encoder = nn.TransformerEncoder(self.encode_layer, num_layers=nlayers)
        
        self.poc_dec = PositionalEncoding(self.embedding_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=nheads)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=nlayers)
        
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, input_att, input_real, hidden = None):
#         print(input_att.shape, input_real.shape)
        seq_len, batch_size = input_real.size()
        
        att_mask = gen_trg_mask(input_att.shape[0])
        real_mask = gen_trg_mask(input_real.shape[0])
        
        ## encoder
        # input_att = replace_start_with_pad(input_real)
        embeds_ = self.embeddings(input_att)
        embeds_ = self.poc_enc(embeds_)
        pad_mask = get_attn_pad_mask(
            input_att.long().transpose(0, 1).contiguous()
        )
        memory = self.transformer_encoder(
            embeds_, 
            mask = att_mask,
            src_key_padding_mask = pad_mask
        )
        
        ## decoder
        embeds = self.embeddings(input_real)
        embeds = self.poc_dec(embeds)
        # print(input_att.shape, input_real.shape, embeds.shape)
        output = self.transformer_decoder(
            ## 
            tgt = embeds, 
            memory = memory, 
            ## 
            tgt_mask = real_mask,
            memory_mask = att_mask,
            ## 
            tgt_key_padding_mask = get_attn_pad_mask(
                input_real.long().transpose(0, 1).contiguous()
            ),
            memory_key_padding_mask = pad_mask,
        )
        
        output = self.linear1(output.view(seq_len*batch_size, -1))
        
        return output, None

In [16]:
# for idx, (i, j) in tqdm.tqdm(enumerate(zip(dataloader_1, dataloader_2))):
#     print(i.shape, j.shape)
#     break

In [17]:
import random
dt[
    random.randint(0, dt.shape[0]-1),
    random.randint(0, dt.shape[1]-1)
].unsqueeze(0).unsqueeze(1)

tensor([[8292]], dtype=torch.int32)

In [18]:
dt.shape[0]

10

In [19]:
random.randint(0, dt.shape[0])

3

In [20]:
def generate(model, input_att, start_words, ix2word, word2ix):     # 给定几个词，根据这几个词生成一首完整的诗歌
    txt = []
    for word in start_words: # 机器学习
        txt.append(word) ## txt: [机,器,学,习]
    print(txt)
    input = Variable(torch.Tensor([word2ix['<START>']]).view(1,1).long())      # tensor([8291.]) → tensor([[8291.]]) → tensor([[8291]])
    print(input)
    hidden = None
    num = len(txt) # 5
    print(num)
    for i in range(48):      # 最大生成长度
        # print(input_att.shape, input.shape)
        output, hidden = model(
            input_att[
                random.randint(0, input_att.shape[0]-1),
                random.randint(0, input_att.shape[1]-1)
            ].unsqueeze(0).unsqueeze(1), 
#             torch.Tensor([[5836]]).long(),
            input, 
            hidden
        )
        # print(output.shape) # [1, 8291]
        if i < num:
            w = txt[i]
            input = Variable(input.data.new([word2ix[w]])).view(1, 1)
        else:
            top_index = output.data[0].topk(1)[1][0]
            # print(top_index)
            w = ix2word[top_index.item()]
            txt.append(w)
            # print(w)
            input = Variable(input.data.new([top_index])).view(1, 1)
        if w == '<EOP>':
            break
    return ''.join(txt)

In [26]:
31072/128

242.75

In [30]:
def train():
    # 模型定义：vocab_size, embedding_dim, hidden_dim —— 8293 * 128 * 256
    modle = Trsfmr_encoder_decoder(len(word2ix), 128, 256)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modle.parameters(), lr=1e-3)  # 学习率1e-3
    loss_meter = meter.AverageValueMeter()

    period = []
    loss2 = []
    for epoch in range(8):     # 最大迭代次数为8
        loss_meter.reset()
        for i, (data_enc, data_dec) in tqdm.tqdm(enumerate(zip(dataloader_1, dataloader_2))):    # data: torch.Size([128, 125]), dtype=torch.int32            
            # data = move_pad_to_tail(data_) # [:, :20]
            
            data_enc = data_enc.long().transpose(0, 1).contiguous()      # long为默认tensor类型，并转置, [125, 10]
            data_dec = data_dec.long().transpose(0, 1).contiguous()
            
            input_real, target = Variable(data_dec[:-1, :]), Variable(data_dec[1:, :])
            output, _ = modle(data_enc[:-1, :], input_real)
            
            loss = criterion(output, target.view(-1))       # torch.Size([15872, 8293]), torch.Size([15872])
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_meter.add(loss.item())    # loss:tensor(3.3510, device='cuda:0', grad_fn=<NllLossBackward>)loss.data:tensor(3.0183, device='cuda:0')

            period.append(i + epoch * len(dataloader))
            loss2.append(loss_meter.value()[0])
            
            if (1 + i) % 2 == 0:       # 每575个batch可视化一次
                print(str(i) +':' + generate(modle, data_enc, '风花雪月', ix2word, word2ix))

        torch.save(modle.state_dict(), 'model_poet_2.pth')
        break ## 我们暂时只跑1个epoch
    plt.plot(period, loss2)
    plt.show()

In [ ]:
if __name__ == '__main__':
    data_path = '../originalDataset/tang.npz'
    data, word2ix, ix2word = get_data()
    data = torch.from_numpy(data)
    dataloader = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱
    train()